# 03 - Confirm Connection to Snowflake

### Import Dependencies

In [1]:
# Snowflake Snowpark
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, StringType, StructType, FloatType, StructField, Variant
from snowflake.snowpark.functions import col, array_construct, udf, call_udf
from snowflake.snowpark.version import VERSION

# misc
import json

### Establish Secure Connection to Snowflake

In [2]:
# Create Snowflake Session object
snowflake_session_prop = json.load(open('session.json'))
session = Session.builder.configs(snowflake_session_prop).create() 

snowflake_environment = session.sql('select current_warehouse(), current_database(), current_schema(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('Warehouse                   : {}'.format(snowflake_environment[0][0]))
print('Database                    : {}'.format(snowflake_environment[0][1]))
print('Schema                      : {}'.format(snowflake_environment[0][2]))
print('Snowflake version           : {}'.format(snowflake_environment[0][3]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))


Warehouse                   : COMPUTE_WH
Database                    : SNOWPARK_OKERA_QUICKSTART
Schema                      : MEDICAL
Snowflake version           : 6.22.0
Snowpark for Python version : 0.7.0


# 04 - Train the Model in Snowflake using Snowpark

### Create a Function to Train the Model

For the purposes of this Quickstart, we'll use a linear regression model.

In [4]:
def train_costs_prediction_model(session: Session, features_table: str) -> Variant:
    import pandas as pd
    import numpy as np
    import sklearn
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error, r2_score
    from sklearn.model_selection import train_test_split

    import os
    from joblib import dump
    
    # Load features from an existing table
    df = session.table(features_table).to_pandas()

    # Specify the feature inputs X and target output y
    X = df.drop('COSTS', axis = 1) 
    y = df['COSTS']

    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.3)

    # Create a linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Save the model to disk then upload to Snowflake stage
    model_output_dir = '/tmp'
    model_file = os.path.join(model_output_dir, 'model.joblib')
    dump(model, model_file)
    session.file.put(model_file, 'MEDICAL_COSTS_STAGE', overwrite=True)
    
    # Return the model R2 score
    return {"R2 on training data ": r2_score(y_train, model.predict(X_train)), "R2 on test data     ": r2_score(y_test, model.predict(X_test))}

### Test the Training Function

In [5]:
# Call the training function to train and test the model
train_costs_prediction_model(session,'COSTS_TRAINING')

{'R2 on training data ': 0.7186327516482559,
 'R2 on test data     ': 0.7647974429033425}

### Create Stored Procedure to Deploy the Training Function to Snowflake

In [6]:
# Simply run this stored procedure to retrain the model as new data is added
session.sproc.register(func=train_costs_prediction_model,
                       name="train_costs_prediction_model",
                       packages=['snowflake-snowpark-python','pandas','numpy','scikit-learn','joblib'],
                       is_permanent=True,
                       stage_location='@MEDICAL_COSTS_STAGE',
                       replace=True)
                       

### Execute the Newly Created Stored Procedure to Train the Model in Snowflake 

In [7]:
print(session.call('train_costs_prediction_model','COSTS_TRAINING'))

{
  "R2 on test data     ": 0.7469180206446722,
  "R2 on training data ": 0.7638595131782806
}


# 05 - Create a User-Defined Function (UDF)

In [8]:
session.clear_imports()
session.clear_packages()

# Add trained model as dependency
session.add_import('@MEDICAL_COSTS_STAGE/model.joblib.gz')
session.add_packages("scikit-learn","pandas","numpy")
    
@udf(name='predict_costs',
     session=session,
     packages=['pandas','joblib','scikit-learn'],
     replace=True,
     is_permanent=True,
     stage_location='@MEDICAL_COSTS_STAGE')
def predict_costs(patient_attributes: list) -> float:
    import sys
    import pandas as pd
    from joblib import load
    
    IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
    import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]
    
    model_file = import_dir + 'model.joblib.gz'
    model = load(model_file)
            
    features = ['AGE','BMI','SEX','CHILDREN','SMOKER','REGION']
    df = pd.DataFrame([patient_attributes], columns=features)

    predicted_costs = model.predict(df)[0]
    return predicted_costs

# 06 - Call User-Defined Function on Training Table

### In Python:

In [9]:
test_df = session.table('COSTS_TRAINING').sample(n=5)

test_df.select ("*", \
                 call_udf("predict_costs", array_construct( \
                                            col("AGE"), \
                                            col("BMI"), \
                                            col("SEX"), \
                                            col("CHILDREN"), \
                                            col("SMOKER"), \
                                            col("REGION")))\
                 .as_("predicted_costs")).show()


--------------------------------------------------------------------------------------------
|"AGE"  |"SEX"  |"BMI"  |"CHILDREN"  |"SMOKER"  |"REGION"  |"COSTS"   |"PREDICTED_COSTS"   |
--------------------------------------------------------------------------------------------
|64     |0      |30.1   |3           |0         |1         |16455.71  |10091.637833690973  |
|64     |0      |39.1   |3           |0         |0         |16085.13  |10498.085463876225  |
|39     |1      |28.3   |1           |1         |0         |21082.16  |26024.726033615076  |
|50     |1      |25.4   |2           |0         |1         |30284.64  |5963.588690117689   |
|35     |0      |23.5   |2           |0         |1         |6402.29   |1672.8857194214816  |
--------------------------------------------------------------------------------------------



### In Snowflake Worksheets as SQL:

Switch to Snowflake Worksheets and enter the following SQL query to call the model as UDF `predict_costs`.

``` SQL
select *, 
    ROUND(predict_costs([AGE,BMI,SEX,CHILDREN,SMOKER,REGION]), 2) as PREDICTED_COSTS, 
    ROUND(PREDICTED_COSTS - COSTS, 2) as DIFFERENCE
    from costs_TRAINING 
    order by DIFFERENCE DESC;
```

![show the UDF in a SQL statement in Worksheets](assets/okera_snowpark_udf_predict_sql.png "Go to Worksheets and run the model as a UDF in SQL")
